In [ ]:
# 1. Install the exact versions that work perfectly together (Nov 2025)
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps xformers==0.0.28 trl peft accelerate bitsandbytes
!pip install -q huggingface_hub transformers sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 148.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 112.4 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch
import json

In [ ]:
from huggingface_hub import login
login("hf_token")

In [ ]:
from huggingface_hub import whoami
print(whoami())  # This shows your username and token details if logged in

{'type': 'user', 'id': '68ecad52b0e099ec94694e09', 'name': 'ayham0010', 'fullname': 'ayham v', 'email': 'ayham.zinedine01@gmail.com', 'emailVerified': True, 'canPay': False, 'billingMode': 'postpaid', 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/2759b8f0a8ea9f2fc3680e57268931a8.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'ClinicalTria_add', 'role': 'write', 'createdAt': '2025-11-27T10:29:35.603Z'}}}


In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    "ayham0010/LungTrial-Eligible",   # or the 4bit version
    load_in_4bit=True,        # keep this True for 4bit merged version, or False for 16bit
    dtype=None,
    max_seq_length=4096,
)
FastLanguageModel.for_inference(model)  # 2x faster inference

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm):

In [ ]:

# =====================================================
# Test function
# =====================================================
def is_eligible(patient_and_trial_dict):
    messages = [
        {"role": "system", "content": "You are an expert in assessing eligibility for oncology clinical trials. Decide whether the patient is eligible and explain your reasoning step by step."},
        {"role": "user", "content": "Decide whether the following patient is eligible for the given clinical trial, and provide reasoning for your decision.\n\nPatient data and trial criteria:\n" + json.dumps(patient_and_trial_dict, indent=2)},
    ]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_new_tokens=512,
        temperature=0.0,      # deterministic = perfect JSON every time
        top_p=0.95,
        do_sample=False
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response.split("assistant")[-1].strip())

In [ ]:
new_test_case = {
    "patient": {
        "age": 71,
        "gender": "M",
        "diagnosis": "stage IV non-small cell lung cancer (adenocarcinoma)",
        "prior_lines": 1,
        "last_chemo_weeks_ago": 6,
        "last_radiation_weeks_ago": 2.5,        # ← tricky!
        "brain_mets": "yes — 2 lesions, treated with SRS 18 months ago, stable on last MRI, asymptomatic",
        "wbc": 3800,
        "platelets": 100000,                   # ← exactly on the cutoff!
        "anc": 1700,
        "bilirubin_mgdl": 1.3,
        "ast_ul": 68,
        "creatinine_mgdl": 1.4,
        "ecog": 2,                             # ← many trials exclude ECOG 2
        "on_anticonvulsants": False,
        "retinal_disease": False,
        "pregnant": False,
        "concurrent_steroids": "dexamethasone 4 mg daily for brain edema (stable dose)"
    },
    "trial_eligibility": {
        "trial_id": None,
        "trial_category": "therapeutic_interventional",
        "age": {"min": 18, "max": None},
        "inclusion": {
            "disease": {
                "cancer_type": "non-small cell lung cancer",
                "stage": "stage IV",
                "measurable_disease_recist": True
            },
            "performance_status": {"scale": "ECOG", "min": 0, "max": 1},   # ← caps at 1!
            "prior_therapy": {
                "max_lines_systemic": 2,
                "washout_weeks": {"chemotherapy": 4, "radiation": 3}
            },
            "brain_metastases": "allowed_if_treated_and_stable",
            "organ_function": {
                "platelets": 100000,
                "anc": 1500,
                "bilirubin_x_uln": 1.5,
                "ast_alt_x_uln": 3.0
            },
            "other_inclusions": ["Stable treated brain metastases allowed", "Stable corticosteroid dose allowed"]
        },
        "exclusion": {
            "active_cns_metastases": True,
            "ecog_greater_than_1": True,           # ← explicit exclusion
            "radiation_within_21_days": True       # ← 3 weeks
        }
    }
}

In [ ]:

# Run the test
print("=== MODEL OUTPUT ON THE CASE ===\n")
is_eligible(new_test_case["patient"] | {"trial_eligibility": new_test_case["trial_eligibility"]})
# (We merge the two sub-dicts because your training data has them under one "input" key)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


=== MODEL OUTPUT ON THE CASE ===

{
  "is_eligible": false,
  "reasoning": "Ineligible: ECOG 2 (required ECOG 0-1), and on steroids (allowed stable dose, but this patient is on a higher dose)."
}
